In [ ]:
import os
import json
from tqdm import tqdm
from glob import glob
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image

import avstack
import avapi

import pickle

In [ ]:
def load_metadata(path_to_results, verbose=False):
    # Load in the metadata
    with open(os.path.join(path_to_results, "launch_metadata.json"), 'r') as f:
        launch_metadata = json.load(f)
    with open(os.path.join(path_to_results, "simulator", "metadata.json"), "r") as f:
        sim_metadata = json.load(f)
    
    # Set up the carla dataset manager
    CSM = avapi.carla.CarlaScenesManager(data_dir=sim_metadata["dataset_path"])
    CDM = CSM.get_scene_dataset_by_index(sim_metadata["scene_idx"])
    all_ts = np.array(CDM.timestamps)
    
    # Get the frame/time mappings
    with open(os.path.join(path_to_results, "simulator", "idx_to_frame_map.txt"), "r") as f:
        idx_to_frame_map = {int(line.rstrip().split()[0]): int(line.rstrip().split()[1]) for line in f.readlines()}
    frame_to_idx_map = {v:k for k, v in idx_to_frame_map.items()}
    
    # Load in the agent paths
    output_paths = {}
    frame_to_output_index = {}
    for agent in ["ego", "command_center"] + [f"agent{i}" for i in range(1, 1+launch_metadata["n_infrastructure_agents"])]:
        output_paths[agent] = {}
        frame_to_output_index[agent] = {}
        for subfolder in ["detections", "tracks"]:
            path_to_sub = os.path.join(path_to_results, agent, subfolder)
            files_in_folder = [os.path.basename(file) for file in sorted(glob(os.path.join(path_to_sub, "*.txt")))]
            output_paths[agent][subfolder] = {}
            frame_to_output_index[agent][subfolder] = {}
            for i, file in enumerate(files_in_folder):
                ts = float(file.split('-')[3].replace('.txt', ''))
                frame = CDM.frames[np.argmin(abs(all_ts - ts))]
                idx = frame_to_idx_map[frame]
                output_paths[agent][subfolder][i] = {
                    "timestamp":ts,
                    "idx": idx,
                    "frame": frame,
                    "path": os.path.join(path_to_sub, file)
                }
                frame_to_output_index[agent][subfolder][frame] = i
            if verbose:
                print("Recognized {} files for agent {} in subfolder {}".format(len(files_in_folder), agent, subfolder))
    return output_paths, frame_to_output_index, CDM

In [ ]:
def _get_base(agent, frame, subfolder, CDM, output_paths, frame_to_output_index, decoder):
    # load up the tracks
    index = frame_to_output_index[agent][subfolder][frame]
    file = output_paths[agent][subfolder][index]["path"]
    with open(file, "rb") as f:
        objects = json.load(f, cls=decoder)

    # replace passive reference with active
    for obj in objects:
        if isinstance(obj.reference, avstack.geometry.PassiveReferenceFrame):
            if obj.reference.frame_id == "ego":
                reference = CDM.get_ego_reference(frame)
            elif "agent" in obj.reference.frame_id:
                sensor = get_lidar_sensor(agent)
                reference = CDM.get_calibration(frame, sensor=sensor).reference
            elif obj.reference.frame_id == "world":
                reference = avstack.geometry.GlobalOrigin3D
            else:
                raise NotImplementedError(obj.reference.frame_id)
            if agent == "command_center":
                obj.state.reference = reference
                obj.state.where_is_t = "bottom"
                obj.state.q.reference = reference
                for member in obj.members:
                    member.reference = reference
                    member.where_is_t = "bottom"
                    member.q.reference = reference
            else:
                if hasattr(obj, "q"):  # HACK for tracks
                    obj.reference = reference
                    obj.where_is_t = "bottom"
                    obj.q.reference = reference
                else:  # HACK for detections
                    obj.box.where_is_t = "bottom"
                    obj.box.position.reference = reference
                    obj.box.attitude.reference = reference

    return objects


def get_detections(agent, frame, CDM, output_paths, frame_to_output_index):
    return _get_base(agent, frame, "detections", CDM, output_paths, frame_to_output_index, avstack.modules.perception.detections.DetectionContainerDecoder)


def get_tracks(agent, frame, CDM, output_paths, frame_to_output_index):
    return _get_base(agent, frame, "tracks", CDM, output_paths, frame_to_output_index, avstack.modules.tracking.tracks.TrackContainerDecoder)


def get_camera_sensor(agent):
    return _get_sensor(agent, "camera")


def get_lidar_sensor(agent):
    return _get_sensor(agent, "lidar")


def _get_sensor(agent, sensor_type):
    if agent == "ego":
        sensor = "main_{}".format(sensor_type)
    elif "agent" in agent:
        if sensor_type == "camera":
            prefix = "CAM"
        else:
            prefix = "LIDAR"
        sensor = "{}_INFRASTRUCTURE_{:03d}".format(prefix, int(agent.replace("agent","")))
    else:
        raise NotImplementedError(agent)
    return sensor


def get_visible_objects_within(agent, frame, CDM, all_agents, max_dist=100):
    # get information
    ego_ref = CDM.get_ego_reference(frame)
    lidar_sensor = get_lidar_sensor(agent)
    camera_sensor = get_camera_sensor(agent)
    
    # get all objects and objects in view
    objects_within_distance = CDM.get_objects_global(frame=frame, max_dist=(ego_ref, max_dist), include_ego=True)
    objects_see_lidar = CDM.get_objects(frame=frame, sensor=lidar_sensor)
    objects_see_all_agents = [CDM.get_objects(frame=frame, sensor=get_lidar_sensor(age)) for age in all_agents]
    all_IDs = {obj.ID for objs in objects_see_all_agents for obj in objs}
    objects_within_distance_visible = np.array([obj for obj in objects_within_distance if obj.ID in all_IDs])

    return objects_within_distance_visible

In [ ]:
def get_all_result_managers(agent, frames, CDM, output_paths, frame_to_output_index, max_dist, all_agents):
    # make all result managers
    res_own_all = []
    res_cc_all = []
    print("Making result managers")
    for frame in tqdm(frames):
        # load objects
        tracks_own = get_tracks(agent=agent, frame=frame, CDM=CDM, output_paths=output_paths, frame_to_output_index=frame_to_output_index)
        tracks_cc = get_tracks(agent="command_center", frame=frame, CDM=CDM, output_paths=output_paths, frame_to_output_index=frame_to_output_index)
        objects_within_distance_visible = get_visible_objects_within(agent, frame, CDM, all_agents, max_dist=max_dist)
        
        # perform assignment to get colors of true/false
        res_own_all.append(avapi.evaluation.ResultManager(
            idx=frame, truths=objects_within_distance_visible, detections=tracks_own, debug=False)
        )
        res_cc_all.append(avapi.evaluation.ResultManager(
            idx=frame, truths=objects_within_distance_visible, detections=tracks_cc, debug=False)
        )
    print("Done")
    return res_own_all, res_cc_all


def get_results_all_agents(CDM, output_paths, frame_to_output_index, all_agents, max_dist=100, idx_frame_start=6, idx_frame_end=-4):
    results = {}
    for agent in all_agents:
        frames = list(frame_to_output_index[agent]["tracks"].keys())[idx_frame_start:idx_frame_end]
        res_own_all, res_cc_all = get_all_result_managers(agent=agent, frames=frames, CDM=CDM, output_paths=output_paths,
                                              frame_to_output_index=frame_to_output_index, max_dist=max_dist, all_agents=all_agents)
        results[agent] = {"own": res_own_all, "cc": res_cc_all}
    return results   


def get_metrics_for_run(results_baseline, all_agents, CDM, output_paths, frame_to_output_index,
                max_dist=100, idx_frame_start=6, idx_frame_end=-4):
    """Gets the metrics for a particular run"""
    metrics = {"CAFPIoB": {}, "CAFNIoB": {}}

    print('Computing metrics for each agent')
    results = get_results_all_agents(CDM, output_paths, frame_to_output_index, all_agents,
                                     max_dist=max_dist, idx_frame_start=idx_frame_start, idx_frame_end=idx_frame_end)
    for agent in all_agents:
        metrics_this_agent = compute_metrics(results_baseline[agent], results[agent], agent)
        if agent == "ego":
            metrics = {**metrics, **metrics_this_agent}
        else:
            metrics["CAFPIoB"][agent] = metrics_this_agent["CAFPIoB"]
            metrics["CAFNIoB"][agent] = metrics_this_agent["CAFNIoB"]
    print('done')
    return metrics


def compute_metrics(results_baseline, results_agent, agent):
    """Compare the candidate results to the baseline to get the increment metrics
    
    Metrics include:
        - CAFP(N)IoB   -- compromised agent false positive/negative increment over baseline
        - ERCCTPIoB    -- ego-relative command center true positive increment over baseline
        - ERCCTPIoE    -- ego-relative command center true positive increment over ego's local information
        - ERCCFP(N)IoB -- ego-relative command center false positive/negative increment over baseline
        - ERCCFP(N)IoE -- ego-relative command center false positive/negative increment over ego's local information
    """
    metrics = {}
    if agent == "ego":
        # ERCCTPIoB
        metrics["ERCCTPIoB"] = np.array([
            res_agent_cc.get_number_of("assignments") - res_base_cc.get_number_of("assignments") \
            for res_agent_cc, res_base_cc in zip(results_agent["cc"], results_baseline["cc"])
        ])
        
        # ERCCTPIoE
        metrics["ERCCTPIoE"] = np.array([
            res_agent_cc.get_number_of("assignments") - res_agent_own.get_number_of("assignments") \
            for res_agent_cc, res_agent_own in zip(results_agent["cc"], results_agent["own"])
        ])
        
        # ERCCFP(N)IoB
        metrics["ERCCFPIoB"] = np.array([
            res_agent_cc.get_number_of("false_positives") - res_base_cc.get_number_of("false_positives") \
            for res_agent_cc, res_base_cc in zip(results_agent["cc"], results_baseline["cc"])
        ])
        metrics["ERCCFNIoB"] = np.array([
            res_agent_cc.get_number_of("false_negatives") - res_base_cc.get_number_of("false_negatives") \
            for res_agent_cc, res_base_cc in zip(results_agent["cc"], results_baseline["cc"])
        ])
        
        # ERCCFP(N)IoE
        metrics["ERCCFPIoE"] = np.array([
            res_agent_cc.get_number_of("false_positives") - res_agent_own.get_number_of("false_positives") \
            for res_agent_cc, res_agent_own in zip(results_agent["cc"], results_agent["own"])
        ])
        metrics["ERCCFNIoE"] = np.array([
            res_agent_cc.get_number_of("false_negatives") - res_agent_own.get_number_of("false_negatives") \
            for res_agent_cc, res_agent_own in zip(results_agent["cc"], results_agent["own"])
        ])
                
    else:
        # CAFP(N)IoB -- do it for all agents and figure out which compromised later
        metrics["CAFPIoB"] = np.array([
            res_agent_cc.get_number_of("false_positives") - res_base_cc.get_number_of("false_positives") \
            for res_agent_cc, res_base_cc in zip(results_agent["cc"], results_baseline["cc"])
        ])
        metrics["CAFNIoB"] = np.array([
            res_agent_cc.get_number_of("false_negatives") - res_base_cc.get_number_of("false_negatives") \
            for res_agent_cc, res_base_cc in zip(results_agent["cc"], results_baseline["cc"])
        ])
    
    return metrics


def make_metrics_table(all_metrics):
    pass

In [ ]:
def plot_and_save_agent_figures(agent, idx_frame, CDM, output_paths, frame_to_output_index, figure_folder, save_prefix="", max_dist=100,
                        all_agents=["ego", "agent1", "agent2", "agent3", "agent4"], **kwargs):
    """Perform analysis and make plots for agents point of view"""
    print(f'Running analysis on {agent}')
    if save_prefix:
        save_prefix += "_"
    
    # get all information from agents and cc
    lidar_sensor = get_lidar_sensor(agent)
    camera_sensor = get_camera_sensor(agent)
    frame = CDM.get_frames(sensor=camera_sensor)[idx_frame]
    img = CDM.get_image(frame=frame, sensor=camera_sensor)
    pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor)
    tracks_own = get_tracks(agent=agent, frame=frame, CDM=CDM, output_paths=output_paths, frame_to_output_index=frame_to_output_index)
    tracks_cc = get_tracks(agent="command_center", CDM=CDM, frame=frame, output_paths=output_paths, frame_to_output_index=frame_to_output_index)
    objects_within_distance_visible = get_visible_objects_within(agent, frame, CDM, all_agents, max_dist=max_dist)
    
    # perform assignment to get colors of true/false
    res_own = avapi.evaluation.ResultManager(idx=frame, truths=objects_within_distance_visible, detections=tracks_own, debug=False)
    res_cc  = avapi.evaluation.ResultManager(idx=frame, truths=objects_within_distance_visible, detections=tracks_cc, debug=False)
    
    # visualize ownship information
    res_img = res_own.visualize(image=img, projection='2d', return_image=True, inline=True, **kwargs)
    res_lid = res_own.visualize(lidar=pc, projection='bev', return_image=True, inline=True, **kwargs)
    im = Image.fromarray(res_img)
    im.save(os.path.join(figure_folder, f"{save_prefix}{agent}_with_own_image.pdf"))
    im = Image.fromarray(res_lid)
    im.save(os.path.join(figure_folder, f"{save_prefix}{agent}_with_own_lidar.pdf"))

    # visualize command center info projected
    res_img = res_cc.visualize(image=img, projection='2d', return_image=True, inline=True, **kwargs)
    res_lid = res_cc.visualize(lidar=pc, projection='bev', return_image=True, inline=True, **kwargs)
    im = Image.fromarray(res_img)
    im.save(os.path.join(figure_folder, f"{save_prefix}{agent}_with_cc_image.pdf"))
    im = Image.fromarray(res_lid)
    im.save(os.path.join(figure_folder, f"{save_prefix}{agent}_with_cc_lidar.pdf"))

    print('done')


def make_and_save_agent_movie(agent, CDM, output_paths, frame_to_output_index, movie_folder, save_prefix="", max_dist=100,
                              all_agents=["ego", "agent1", "agent2", "agent3", "agent4"], 
                              idx_frame_start=6, idx_frame_end=-4, **kwargs):
    """Generate and save the movie from the agent's execution"""
    print(f'Generating movie for {agent}')
    if save_prefix:
        save_prefix += "_"

    # load information
    lidar_sensor = get_lidar_sensor(agent)
    camera_sensor = get_camera_sensor(agent)         
    frames = list(frame_to_output_index[agent]["tracks"].keys())[idx_frame_start:idx_frame_end]
    print("loading agent images")
    all_imgs = [CDM.get_image(frame=frame, sensor=camera_sensor) for frame in tqdm(frames)]
    res_own_all, res_cc_all = get_all_result_managers(agent, frames, CDM, output_paths,
                                                      frame_to_output_index, max_dist, all_agents)
    
    # make all result managers
    res_own_all = []
    res_cc_all = []
    print("Making result managers")
    for frame in tqdm(frames):
        # load objects
        tracks_own = get_tracks(agent=agent, frame=frame, CDM=CDM, output_paths=output_paths, frame_to_output_index=frame_to_output_index)
        tracks_cc = get_tracks(agent="command_center", frame=frame, CDM=CDM, output_paths=output_paths, frame_to_output_index=frame_to_output_index)
        objects_within_distance_visible = get_visible_objects_within(agent, frame, CDM, all_agents, max_dist=max_dist)
        
        # perform assignment to get colors of true/false
        res_own_all.append(avapi.evaluation.ResultManager(
            idx=frame, truths=objects_within_distance_visible, detections=tracks_own, debug=False)
        )
        res_cc_all.append(avapi.evaluation.ResultManager(
            idx=frame, truths=objects_within_distance_visible, detections=tracks_cc, debug=False)
        )
    print("Done")
    
    # make and save movies
    filename = os.path.join(movie_folder, f"{save_prefix}{agent}_with_own")
    avapi.visualize.movie.make_movie(
        raw_imgs=all_imgs, boxes=res_own_all, save=True, name=filename, show_in_notebook=False
    )
    filename = os.path.join(movie_folder, f"{save_prefix}{agent}_with_cc")
    avapi.visualize.movie.make_movie(
        raw_imgs=all_imgs, boxes=res_cc_all, save=True, name=filename, show_in_notebook=False
    )
    print('done')

In [ ]:
def gen_figures_over_folder(end_folder, idx_frame_static, all_agents=["ego", "agent1", "agent2", "agent3", "agent4"]):
    # set paths
    path_to_results = os.path.join(base_path, end_folder)
    
    # make folders
    figure_folder = os.path.join("figures", end_folder)
    movie_folder = os.path.join("movies", end_folder)
    os.makedirs(movie_folder, exist_ok=True)
    os.makedirs(figure_folder, exist_ok=True)
    
    # get path information
    output_paths, frame_to_output_index, CDM = load_metadata(path_to_results)
    
    # generate results figures
    for agent in all_agents:
        plot_and_save_agent_figures(agent, idx_frame=idx_frame_static, save_prefix=end_folder,
                                    CDM=CDM, figure_folder=figure_folder, output_paths=output_paths,
                                    frame_to_output_index=frame_to_output_index, show=False)
    
    # generate results movies
    for agent in all_agents:
        make_and_save_agent_movie(agent, CDM, output_paths, frame_to_output_index, movie_folder=movie_folder)

## Generate Figures and Movies

In [ ]:
gen_figures_over_folder(end_folder="coordinated_2_attackers", idx_frame_static=40)

In [ ]:
base_path = "../../../../all_scenarios/"
end_folders = sorted(next(os.walk(base_path))[1])
idx_frame_static = 30
print("Running analysis over the following subfolders:\n[\n{}\n]\n".format(",\n".join(end_folders)))

for i, end_folder in enumerate(end_folders):
    print_str = "Running {} / {}: {}".format(i+1, len(end_folders), end_folder).upper()
    n_dash = len(print_str) + 6
    print("-"*n_dash)
    print(print_str)
    print("-"*n_dash)
    gen_figures_over_folder(end_folder, idx_frame_static)

## Run Metrics Analysis

In [ ]:
results_baseline = None

In [ ]:
# Set up all folders
base_path = "../../../../all_scenarios/"
end_folders = sorted(next(os.walk(base_path))[1])
print("Running metrics generation over the following subfolders:\n[\n{}\n]\n".format(",\n".join(end_folders)))
all_agents=["ego", "agent1", "agent2", "agent3", "agent4"]
# all_agents = ["ego", "agent1"]
metrics_all = {folder: {} for folder in end_folders}
idx_frame_start = 6
idx_frame_end = -10

# ensure that "baseline" is at the beginning
base_folder = [folder for folder in end_folders if "baseline" in folder]
end_folders = base_folder + [folder for folder in end_folders if "baseline" not in folder]

# run all other results as increments over baseline
for i, end_folder in enumerate(end_folders):
    # load metadata
    path_to_results = os.path.join(base_path, end_folder)
    output_paths, frame_to_output_index, CDM = load_metadata(path_to_results)
    with open(os.path.join(path_to_results, "launch_metadata.json"), 'r') as f:
        launch_metadata = json.load(f)
    metrics_all[end_folder]["launch_metadata"] = launch_metadata
    with open(os.path.join(path_to_results, "simulator", "metadata.json"), "r") as f:
        sim_metadata = json.load(f)
    metrics_all[end_folder]["sim_metadata"] = sim_metadata

    # get metrics
    if "baseline" in end_folder:
        if results_baseline is None:
            print("getting baseline results".upper())
            results_baseline = get_results_all_agents(CDM, output_paths, frame_to_output_index, all_agents)
        else:
            print("already obtained baseline results...")
    else:
        print_str = "Running {} / {}: {}".format(i, len(end_folders)-1, end_folder).upper()
        n_dash = len(print_str) + 6
        print("-"*n_dash)
        print(print_str)
        print("-"*n_dash)
        metrics_all[end_folder]["metrics"] = get_metrics_for_run(
            results_baseline=results_baseline, all_agents=all_agents, CDM=CDM,
            output_paths=output_paths, frame_to_output_index=frame_to_output_index,
            idx_frame_start=idx_frame_start, idx_frame_end=idx_frame_end,
        )

# save all metrics
with open('all_metrics.p', 'wb') as f:
    pickle.dump(metrics_all, f)
print('saved metrics')

In [ ]:
# save all metrics
with open('all_metrics.p', 'wb') as f:
    pickle.dump(metrics_all, f)
print('saved metrics')